In [ ]:
import numpy as np
import os
dtype = np.float32

In [ ]:
import torch
import torchvision
# import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

In [ ]:
import json
import os
path = "/home/jgryu/Weight_compression/Wparam_dataset/path_json"
list = os.listdir(path)

for l in list :
    file_path = os.path.join(path, l)
    with open(file_path, 'r') as file:
        data = json.load(file)
        
    for i in range(len(data)):
        data[i] = data[i].replace("/home/jgryu/Weight_compression", "")
        
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# V4
weight tensor splice를 각각 npy 파일로 저장

In [ ]:
# import json, os
# import numpy as np
# import tqdm
# dataset_folder = "/home/jgryu/Weight_compression/Wparam_dataset"
# param_type = 'embed'
# split = ['train', 'val']

# for s in split:
#     print(f'Start {s}')
    
#     path = os.path.join(dataset_folder, 'path_json', f'{param_type}_tensor_path_{s}.json')
#     with open(path, 'r') as f:
#         tensor_path_list = json.load(f)
    
#     count = 0
#     for tensor_path in tqdm.tqdm(tensor_path_list):
        
     
#         directory = dataset_folder + tensor_path

#         # 디렉토리 삭제
#         if os.path.exists(directory):
#             os.remove(directory)
#             print("디렉토리가 삭제되었습니다.")
#         else:
#             print("디렉토리가 존재하지 않습니다.")

In [ ]:
import json, os
import numpy as np
import tqdm
dataset_folder = "/home/jgryu/Weight_compression/Wparam_dataset"
param_type = 'mlp'
split = ['train', 'val']
dim = 128
param_type = 'attn'

def check_contains_any_substrings(string, substrings):
    return any(substring in string for substring in substrings)

def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b', '13b', 'mini', 'small']
model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b','mini']
# model_filter2 = ['llama', '7b']

for s in split:
    print(f'Start {s}')
    
    path = os.path.join(dataset_folder, 'path_json', f'{param_type}_tensor_path_{s}.json')
    with open(path, 'r') as f:
        tensor_path_list = json.load(f)
        
    save_path = os.path.join(dataset_folder, 'tensor_slices', s)
    os.makedirs(save_path, exist_ok=True)
    
    count = 0
    
    filtered_tensor_paths = [
        tensor_path for tensor_path in tqdm.tqdm(tensor_path_list)
        if check_contains_any_substrings(tensor_path.lower(), model_filter)
        and contains_all_substrings(tensor_path.lower(), model_filter2)
    ]
    
    for tensor_path in tqdm.tqdm(filtered_tensor_paths):
    # for tensor_path in tqdm.tqdm(tensor_path_list):
    #     if not check_contains_any_substrings(tensor_path.lower(), model_filter):
    #         continue
    #     if not contains_all_substrings(tensor_path.lower(), model_filter2):
    #         continue
        
        path = os.path.join(save_path,  tensor_path.replace("/model_param_tensor/" ,"").replace(".npy", ""), f'd={dim}')
        if os.path.isdir(path):
            # print('continue')
            continue
        try:
            t = np.load(dataset_folder + tensor_path)
        except Exception as e:
            print(f"An error occurred: {e}")
            print(f"Fail load {path}")
            continue

        if t.size % (dim) != 0:
            continue 
            # print(f'나누어 떨어지지 않습니다.')
        t = t.reshape(-1, dim)
        l = t.shape[0]

        for i in range(l):
            os.makedirs(path, exist_ok=True)
            np.save(os.path.join(path, f'd={dim}_{i}.npy'), t[i])
        count += l

Start train


100%|██████████| 3212/3212 [1:12:05<00:00,  1.35s/it]


Start val


 41%|████      | 326/804 [13:57<20:27,  2.57s/it]  


OSError: [Errno 28] No space left on device: '/home/jgryu/Weight_compression/Wparam_dataset/tensor_slices/val/meta-llama/CodeLlama-7b-Python-hf/model-layers-30-self_attn-q_proj-weight/d=128/d=128_46253.npy'

In [ ]:
# import os
# import shutil
# from pathlib import Path

# def reorganize_npy_files(base_path: str):
#     """
#     Wparam_dataset/tensor_slices/train/ 경로 아래의 모든 d=1024_{n}.npy 파일들을
#     해당하는 d=1024 디렉토리로 이동시키는 함수
    
#     Args:
#         base_path (str): 기본 경로 (Wparam_dataset/tensor_slices/train/)
#     """
#     # 기본 경로를 Path 객체로 변환
#     base = Path(base_path)
    
#     # 모든 .npy 파일 찾기
#     for npy_file in base.rglob("*.npy"):
#         # 파일명이 'd=1024_'로 시작하는지 확인
#         if npy_file.name.startswith("d=1024_"):
#             # 현재 파일의 디렉토리
#             current_dir = npy_file.parent
            
#             # 새로운 디렉토리 경로 생성 (d=1024 서브디렉토리)
#             new_dir = current_dir / "d=1024"
            
#             # 새 디렉토리가 없으면 생성
#             if not new_dir.exists():
#                 new_dir.mkdir(parents=True, exist_ok=True)
            
#             # 새로운 파일 경로
#             new_file_path = new_dir / npy_file.name
            
#             try:
#                 # 파일 이동
#                 shutil.move(str(npy_file), str(new_file_path))
#                 print(f"Moved: {npy_file} -> {new_file_path}")
#             except Exception as e:
#                 print(f"Error moving {npy_file}: {e}")


# base_path = "Wparam_dataset/tensor_slices/train/"

# # 경로가 존재하는지 확인
# if not os.path.exists(base_path):
#     print(f"Error: Path {base_path} does not exist")
#     return

# # 파일 재구성 실행
# reorganize_npy_files(base_path)
# print("File reorganization completed")


In [ ]:
import glob
import random
import json
def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

save_path = '/home/jgryu/Weight_compression/model_param_dataset/'
dataset_path = '/home/jgryu/Weight_compression/model_param_tensor/'

tenosr_path_list = glob.glob(f'{dataset_path}/**/*.npy', recursive=True)

print(f'all : {len(tenosr_path_list)}')

data = {'attn':[], 'mlp':[], 'embed':[], 'layernorm':[]}   

random.seed(100)

for weight_type in ['attn', 'mlp', 'embed', 'layernorm']: 
    for path in tenosr_path_list:
        if contains_all_substrings(path.lower(), [weight_type]):
            data[weight_type].append(path)
    print(f'{weight_type} : {len(data[weight_type])}')


    random.shuffle(data[weight_type])

    split_index = int(0.8 * len(data[weight_type]))
    train_data = data[weight_type][:split_index]
    validation_data = data[weight_type][split_index:]

    file_path_train = os.path.join(save_path, f'{weight_type}_tensor_path_train.json')
    file_path_validation = file_path_train.replace('train', 'val')

    with open(file_path_train, 'w') as json_file:
        json.dump(train_data, json_file, indent=4)
    with open(file_path_validation, 'w') as json_file:
        json.dump(validation_data, json_file, indent=4)

# V3
각 tensor의 path를 train val split해서 json 파일로 저장

In [ ]:
import glob
import random
import json
def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

save_path = '/home/jgryu/Weight_compression/model_param_dataset/'
dataset_path = '/home/jgryu/Weight_compression/model_param_tensor/'

tenosr_path_list = glob.glob(f'{dataset_path}/**/*.npy', recursive=True)

print(f'all : {len(tenosr_path_list)}')

data = {'attn':[], 'mlp':[], 'embed':[], 'layernorm':[]}   

random.seed(100)

for weight_type in ['attn', 'mlp', 'embed', 'layernorm']: 
    for path in tenosr_path_list:
        if contains_all_substrings(path.lower(), [weight_type]):
            data[weight_type].append(path)
    print(f'{weight_type} : {len(data[weight_type])}')


    random.shuffle(data[weight_type])

    split_index = int(0.8 * len(data[weight_type]))
    train_data = data[weight_type][:split_index]
    validation_data = data[weight_type][split_index:]

    file_path_train = os.path.join(save_path, f'{weight_type}_tensor_path_train.json')
    file_path_validation = file_path_train.replace('train', 'val')

    with open(file_path_train, 'w') as json_file:
        json.dump(train_data, json_file, indent=4)
    with open(file_path_validation, 'w') as json_file:
        json.dump(validation_data, json_file, indent=4)

# V2
state_dict tensor별로 저장

In [4]:
import numpy as np
import os
dtype = np.float32

import torch
import torchvision
# import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

def get_ckpt_path(path, branch = 'main'):
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

def check_contains_any_substrings(string, substrings):
    return any(substring in string for substring in substrings)

model_zoo_path = '/home/jgryu/Weight_compression/Wparam_dataset/model_zoo/huggingface'
# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b', '13b', 'mini', 'small']
# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b','mini']
# model_filter = ['-13b', '-small']
model_filter = ['meta-llama-3-8b', 'meta-llama-3.1-8b']

model_list = os.listdir(model_zoo_path)
print()
ckpt_path_list = []
for ck in model_list:
    if check_contains_any_substrings(ck.lower(), model_filter):
        ckpt_path = get_ckpt_path(os.path.join(model_zoo_path, ck))
        if ckpt_path is not None:
            ckpt_path_list.append(ckpt_path)

print(len(ckpt_path_list))

save_dir = '/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor'

for ckpt_path in ckpt_path_list:
    model_name = ckpt_path.split('/')[-3]
    print(model_name)
    model_name = model_name.split('--')
    
    save_path = os.path.join(save_dir, model_name[1], model_name[2])

    if os.path.isdir(save_path) and bool(os.listdir(save_path)):
        print('### Skip ###')
        continue

    try:
        model = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True, trust_remote_code=True)
        
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        state_dict = model.state_dict()
        for k, v in state_dict.items():
            np.save(os.path.join(save_path, k.replace(".", "-")), v)
            
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Fail load model from {ckpt_path}")




4
models--meta-llama--Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.09s/it]


models--meta-llama--Meta-Llama-3-8B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.20s/it]


models--meta-llama--Meta-Llama-3.1-8B


Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.08s/it]


models--meta-llama--Meta-Llama-3-8B


Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.54s/it]


In [3]:
len(model_list)

201

In [ ]:
def get_ckpt_path(path, branch = 'main'):
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

def check_contains_any_substrings(string, substrings):
    return any(substring in string for substring in substrings)

In [ ]:
model_zoo_path = '/home/jgryu/Weight_compression/model_zoo/huggingface'
# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b', '13b', 'mini', 'small']
model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b','mini']
model_list = os.listdir(model_zoo_path)
ckpt_path_list = []
for ck in model_list:
    if check_contains_any_substrings(ck.lower(), model_filter):
        ckpt_path = get_ckpt_path(os.path.join(model_zoo_path, ck))
        if ckpt_path is not None:
            ckpt_path_list.append(ckpt_path)

In [ ]:
save_dir = '/home/jgryu/Weight_compression/model_param_tensor'

for ckpt_path in ckpt_path_list:
    try:
        model = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Fail load model from {ckpt_path}")
        continue
    
    model_name = ckpt_path.split('/')[-3]
    print(model_name)
    model_name = model_name.split('--')
    
    save_path = os.path.join(save_dir, model_name[1], model_name[2])
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    if os.path.isdir(save_path) and not os.listdir(save_path):
        continue

    state_dict = model.state_dict()
    for k, v in state_dict.items():
        np.save(os.path.join(save_path, k.replace(".", "-")), v)

# Version 1

In [ ]:
import numpy as np
import os

model_filter = ['llama-2-7b-hf', 'attn']
dim = [128, 256, 512]
for d in dim:
    print("##### Start generating WP dataset #####")
    path_save = '/home/jgryu/Weight_compression/Wparam_dataset/Wparam_npy/'
    dataset_path = '/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor/meta-llama'

    import glob
    def contains_all_substrings(string, substrings):
        return all(substring in string for substring in substrings)

    np_list = []
    tenosr_path_list = glob.glob(f'{dataset_path}/**/*.npy', recursive=True)
    for path in tenosr_path_list:
        if contains_all_substrings(path.lower(), model_filter):
            np_tensor = np.load(path).astype('float32')
            if np_tensor.size % d != 0:
                continue
            np_list.append(np_tensor.reshape(-1,d))
            
    np_list = np.vstack(np_list)
    indices = np.random.permutation(np_list.shape[0])
    np_list = np_list[indices]
    split_index = int(0.8 * np_list.shape[0])
    train_data = np_list[:split_index]
    validation_data = np_list[split_index:]

    np.save(path_save + '_'.join(model_filter) + f'_d={d}_train', train_data)
    np.save(path_save + '_'.join(model_filter) + f'_d={d}_val', validation_data)
    
    print("##### End generating WP dataset #####")

In [ ]:
import numpy as np
import os
dtype = np.float32
import torch
import torchvision
# import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir
import argparse

def ckpt_path(path, branch = 'main'):
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_filter', nargs='+', type=str, default=['gemma', '2b'], 
                        help="Model filter list")
    parser.add_argument('--d', type=int, default=128, 
                        help="Dimension size")
    parser.add_argument('--path_model_zoo', type=str, default='/home/jgryu/Weight_compression/model_zoo/huggingface',
                        help="Path to the model zoo")
    return parser.parse_args()

args = parse_args()

ckpt_list = os.listdir(args.path_model_zoo)
path_list_ckpts = []
for ck in ckpt_list :
    if contains_all_substrings(ck.lower(), args.model_filter):
        path_ckpt = ckpt_path(os.path.join(args.path_model_zoo, ck))
        if path_ckpt is not None:
            path_list_ckpts.append(path_ckpt)

for p in path_list_ckpts:
    print(p.split('/')[-3])
    
data = {'attn':[], 'mlp':[], 'embed':[], 'layernorm':[]}   

for path_model in path_list_ckpts:
    try:
        model = AutoModelForCausalLM.from_pretrained(path_model, local_files_only=True)
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Fail load model from {path_model}")
        continue
    
    state_dict = model.state_dict()
    for k, v in state_dict.items():
        # print(k)
        # print(v.shape)
        if v.numel() % args.d != 0:
            print(f'나누어 떨어지지 않습니다.')
            continue
        if 'attn' in k.lower():
            v_split = v.view(-1, args.d).numpy()
            data['attn'].append(v_split)
        elif 'mlp' in k.lower():
            v_split = v.view(-1, args.d).numpy()
            data['mlp'].append(v_split)
        elif 'embed' in k.lower():
            v_split = v.view(-1, args.d).numpy()
            data['embed'].append(v_split)
        elif 'layernorm' in k.lower():
            v_split = v.view(-1, args.d).numpy()
            data['layernorm'].append(v_split)

path_save = '/home/jgryu/Weight_compression/model_parm_dataset/'

print('## Shape ##')
for k, v in data.items():
    data[k] = np.vstack(v)
    print(k, ': ', data[k].shape)

# attn = np.vstack(attn)
# mlp = np.vstack(mlp)
# embed = np.vstack(embed)
# layernorm = np.vstack(layernorm)

# print('attn :', attn.shape)
# print('mlp :', mlp.shape)
# print('embed :', embed.shape)
# print('layernorm :', layernorm.shape)

# 데이터를 랜덤하게 섞기 위한 인덱스 생성
    indices = np.random.permutation(data[k].shape[0])
    shuffled_data = data[k][indices]
    # shuffled_data = data[k]
    split_index = int(0.8 * shuffled_data.shape[0])
    train_data = shuffled_data[:split_index]
    validation_data = shuffled_data[split_index:]

    np.save(path_save + '_'.join(args.model_filter) + '_' + k + f'_d={args.d}_train', train_data)
    np.save(path_save + '_'.join(args.model_filter) + '_' + k + f'_d={args.d}_validation', validation_data)


# path_save = '/home/jgryu/Weight_compression/model_parm_dataset/'
# np.save(path_save + '_'.join(model_filter) + '_attn' + f'_d={d}', attn)
# np.save(path_save + '_'.join(model_filter) + '_mlp' + f'_d={d}', mlp)
# np.save(path_save + '_'.join(model_filter) + '_embed' + f'_d={d}', embed)
# np.save(path_save + '_'.join(model_filter) + '_layernorm' + f'_d={d}', layernorm)
# print("데이터가 '.npy' 파일로 저장되었습니다.")


# print("Train set shape:", train_data.shape)
# print("Validation set shape:", validation_data.shape)


In [5]:
import glob
import numpy as np
file_glob = '/home/jgryu/Weight_compression/Wparam_dataset/tensor_slices/train/meta-llama/CodeLlama-7b-hf/model-layers-0-self_attn-k_proj-weight'
files = sorted(glob.glob(file_glob + '/**/*.npy', recursive=True))
# print(files)

print(f'length{len(files)}=======')
if not files:
    raise RuntimeError(f"No images found with glob '{file_glob}'.")

l = []
for f in files:
    l.append(np.load(f))
n = np.vstack(l)
np.save('/home/jgryu/Weight_compression/Wparam_dataset/npy/test', n)

length131072=======


In [6]:
n.shape

(131072, 128)